In [8]:
import torch

B = 1
C = 3
H = 4
W = 4

patch_number = 2

# mask = torch.randint(0, 2, (B, 1, H, W)).float()
# img = torch.range(0, B*C*H*W-1).reshape(B, H, W, C).float()

mask = torch.Tensor(
    [                               # B
        [                           # 1
            [                       # H
                [1., 1., 0., 1.],   # W
                [1., 0., 0., 0.],
                [1., 1., 1., 1.],
                [1., 0., 0., 0.]
            ]
        ]
    ]
)

# 掩码分块
H_patch = H // patch_number
W_patch = W // patch_number
mask_patches = mask.reshape(
    B, patch_number, H_patch, patch_number, W_patch
).permute(
    0, 1, 3, 2, 4
).reshape(
    B, -1, H_patch*W_patch
)      

# 图像分块
img_patches = img.reshape(
    B, patch_number, H_patch, patch_number, W_patch, C
).permute(
    0, 1, 3, 2, 4, 5
).reshape(
    B, -1, H_patch*W_patch, C
)             

# 分块均值池化
window_avg_pooling = torch.einsum('bsl,bsld->bsd', mask_patches, img_patches)

print('mask:\n', mask)
print('mask_patches:\n', mask_patches)

print('window_avg_pooling:\n', window_avg_pooling)

print('img:\n', img)
print('img_patches:\n', img_patches)

torch.Size([1, 4, 4])
torch.Size([1, 4, 4, 3])
mask:
 tensor([[[[1., 1., 0., 1.],
          [1., 0., 0., 0.],
          [1., 1., 1., 1.],
          [1., 0., 0., 0.]]]])
mask_patches:
 tensor([[[1., 1., 1., 0.],
         [0., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 0., 0.]]])
window_avg_pooling:
 tensor([[[15., 18., 21.],
         [ 9., 10., 11.],
         [87., 90., 93.],
         [63., 65., 67.]]])
img:
 tensor([[[[ 0.,  1.,  2.],
          [ 3.,  4.,  5.],
          [ 6.,  7.,  8.],
          [ 9., 10., 11.]],

         [[12., 13., 14.],
          [15., 16., 17.],
          [18., 19., 20.],
          [21., 22., 23.]],

         [[24., 25., 26.],
          [27., 28., 29.],
          [30., 31., 32.],
          [33., 34., 35.]],

         [[36., 37., 38.],
          [39., 40., 41.],
          [42., 43., 44.],
          [45., 46., 47.]]]])
img_patches:
 tensor([[[[ 0.,  1.,  2.],
          [ 3.,  4.,  5.],
          [12., 13., 14.],
          [15., 16., 17.]],

        

/tmp/ipykernel_41227/2929050846.py:11: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  img = torch.range(0, B*C*H*W-1).reshape(B, H, W, C).float()


In [13]:
# inf的四则运算规则测试
import torch

a = torch.tensor([-1.2], dtype=torch.float32)
b = torch.tensor([torch.inf], dtype=torch.float32)

print(a + b)
print(a - b)
print(a * b)
print(a / b)
print(torch.exp(b))
print(torch.exp(-b))

tensor([inf])
tensor([-inf])
tensor([-inf])
tensor([-0.])
tensor([inf])
tensor([0.])


In [3]:
# cat 与 stack的深度复制测试
import torch

a = torch.tensor([1, 2, 3]).unsqueeze(0)
b = torch.cat([a, a], dim=0)

a[0][0] = 5
print(a)
print(b)

tensor([[5, 2, 3]])
tensor([[1, 2, 3],
        [1, 2, 3]])


In [6]:
# 使用for循环构造列表的测试

import torch

a = torch.tensor([1, 2])
b = torch.tensor([3, 4])

print([
    torch.stack((a, b), dim=0) for i in range(10)
])

[tensor([[1, 2],
        [3, 4]]), tensor([[1, 2],
        [3, 4]]), tensor([[1, 2],
        [3, 4]]), tensor([[1, 2],
        [3, 4]]), tensor([[1, 2],
        [3, 4]]), tensor([[1, 2],
        [3, 4]]), tensor([[1, 2],
        [3, 4]]), tensor([[1, 2],
        [3, 4]]), tensor([[1, 2],
        [3, 4]]), tensor([[1, 2],
        [3, 4]])]


In [8]:
for i, (x, y) in enumerate(zip([1, 2, 3], [4, 5, 6])):
    print(i, x, y)

0 1 4
1 2 5
2 3 6


In [4]:
# 模型测试
import torch
import torch.nn as nn

net = nn.Linear(4, 3, bias=True)
data = torch.randn((16, 16, 4), dtype=torch.float32)

out = net(data)
print(out.shape)

print((*data.shape[:2], ))

torch.Size([16, 16, 3])
(16, 16)


In [1]:
# DCAMA代码测试
from functools import reduce
from operator import add
import torch

feat_channels = [128, 256, 512, 1024]
nlayers = [2, 2, 18, 2]

lids = reduce(add, [[i + 1] * x for i, x in enumerate(nlayers)])
stack_ids = torch.tensor(lids).bincount()[-4:].cumsum(dim=0)
print(lids)
print(stack_ids)
'''
DCAMA中，swin-b骨干网络的输出特征图情况如下（输入图片尺寸为[2, 3, 384, 384]）:
torch.Size([2, 9216, 128])
torch.Size([2, 9216, 128])

torch.Size([2, 2304, 256])
torch.Size([2, 2304, 256])

torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])
torch.Size([2, 576, 512])

torch.Size([2, 144, 1024])
torch.Size([2, 144, 1024])
'''

[1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4]
tensor([ 2,  4, 22, 24])


In [2]:
# torch.max方法测试
import torch

a = torch.randn((2, 3, 4, 4), dtype=torch.float32)
b = torch.max(a, dim=0, keepdim=True)
c = b.values
print('b:\n', b)

print('c:\n', c)

b:
 torch.return_types.max(
values=tensor([[[[-0.6337,  0.7852, -0.6930,  0.2895],
          [ 1.6348,  0.8421,  2.3349,  0.7589],
          [ 1.3289,  1.3033, -0.0552, -0.0970],
          [ 0.5299, -0.2014,  1.4027, -0.1976]],

         [[ 1.2400, -0.8829, -0.3356,  2.1629],
          [ 1.0375,  0.8535, -0.0611,  0.4860],
          [ 1.6303,  0.5101, -0.7879,  0.5074],
          [ 1.1449,  1.7214,  1.4532,  1.9156]],

         [[ 0.2400,  1.3669,  1.1057,  1.1713],
          [ 0.6118,  1.2392,  2.4902,  1.2335],
          [-0.3423,  0.9491,  1.2006,  1.9659],
          [ 0.3903,  0.9943,  0.1393,  1.6231]]]]),
indices=tensor([[[[0, 1, 1, 1],
          [0, 1, 1, 0],
          [1, 1, 1, 0],
          [1, 1, 0, 1]],

         [[0, 0, 1, 1],
          [0, 1, 0, 0],
          [1, 0, 1, 1],
          [1, 1, 1, 0]],

         [[1, 1, 0, 1],
          [1, 1, 0, 1],
          [1, 0, 1, 0],
          [1, 0, 1, 0]]]]))
c:
 tensor([[[[-0.6337,  0.7852, -0.6930,  0.2895],
          [ 1.6348,  0.84

In [12]:
# tensor切片测试
import torch

a = torch.randint(0, 10, (3, 1, 2, 2))
print(a)

print(torch.cat((a[1:],a[:1]), dim=0))


b = [1, 2, 3]
print(b[1:] + b[:1])

tensor([[[[3, 8],
          [1, 4]]],


        [[[8, 9],
          [3, 5]]],


        [[[5, 2],
          [0, 7]]]])
tensor([[[[8, 9],
          [3, 5]]],


        [[[5, 2],
          [0, 7]]],


        [[[3, 8],
          [1, 4]]]])
[2, 3, 1]


In [11]:
# DCAMA权重文件转换

import torch

checkpoint_file = '/home/hjf/workspace/mmsegmentation/work_dirs/dcama_pretrained/pascal-5i/swin_fold0.pt'
# checkpoint_file = '/home/hjf/workspace/mmsegmentation/work_dirs/dcama_pretrained/coco-20i/swin_fold0.pt'

params = torch.load(checkpoint_file, map_location='cpu')
for key in params:
    # if 'backbone' in key:
    #     continue
    print('{:70s}, {}'.format(key, params[key].shape))


module.backbone.patch_embed.proj.weight                               , torch.Size([128, 3, 4, 4])
module.backbone.patch_embed.proj.bias                                 , torch.Size([128])
module.backbone.patch_embed.norm.weight                               , torch.Size([128])
module.backbone.patch_embed.norm.bias                                 , torch.Size([128])
module.backbone.layers.0.blocks.0.norm1.weight                        , torch.Size([128])
module.backbone.layers.0.blocks.0.norm1.bias                          , torch.Size([128])
module.backbone.layers.0.blocks.0.attn.relative_position_bias_table   , torch.Size([529, 4])
module.backbone.layers.0.blocks.0.attn.relative_position_index        , torch.Size([144, 144])
module.backbone.layers.0.blocks.0.attn.qkv.weight                     , torch.Size([384, 128])
module.backbone.layers.0.blocks.0.attn.qkv.bias                       , torch.Size([384])
module.backbone.layers.0.blocks.0.attn.proj.weight                    , torch.

In [6]:
# tensor索引测试

import torch

label = torch.tensor([
    [1, 0, 1, 1],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [1, 1, 0, 0]
]).reshape(
    1, 2, 2, 2, 2
).permute(
    0, 1, 3, 2, 4
).reshape(1, 4, 4)

label_fg = label
label_bg = 1 - label
print('label_fg\n', label_fg)
print('label_bg\n', label_bg)

label_fg_sum = label_fg.sum(dim=-1)
label_bg_sum = label_bg.sum(dim=-1)
print('label_fg_sum\n', label_fg_sum)
print('label_bg_sum\n', label_bg_sum)

label_fg_sum[label_fg_sum<label_bg_sum] = 1e-6
label_bg_sum[label_fg_sum>=label_bg_sum] = 1e-6
print('label_fg_sum\n', label_fg_sum)
print('label_bg_sum\n', label_bg_sum)

label_fg[label_fg_sum<1] *= 0
label_bg[label_bg_sum<1] *= 0
print('label_fg\n', label_fg)
print('label_bg\n', label_bg)

label_fg
 tensor([[[1, 0, 0, 1],
         [1, 1, 0, 0],
         [0, 0, 1, 1],
         [1, 0, 0, 0]]])
label_bg
 tensor([[[0, 1, 1, 0],
         [0, 0, 1, 1],
         [1, 1, 0, 0],
         [0, 1, 1, 1]]])
label_fg_sum
 tensor([[2, 2, 2, 1]])
label_bg_sum
 tensor([[2, 2, 2, 3]])
label_fg_sum
 tensor([[2, 2, 2, 0]])
label_bg_sum
 tensor([[0, 0, 0, 3]])
label_fg
 tensor([[[1, 0, 0, 1],
         [1, 1, 0, 0],
         [0, 0, 1, 1],
         [0, 0, 0, 0]]])
label_bg
 tensor([[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 1, 1, 1]]])
